In [11]:
import os 
import sys
import logging
import re 
import datetime
import pickle
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
sys.path.append('/home/malcolm/WorkTransit/scripts/')

## Starts here

In [14]:
import WorkTransitLib.RawDirections as rd
import WorkTransitLib.parseRawResponse as parser

In [15]:
now = datetime.datetime.now()
print(now)
time_hour = now.time().hour 
weekday = now.weekday()

2022-06-24 17:14:06.630342


In [16]:
time_hour = 18
weekday = 5

In [17]:
# Weekend Morning 
if (weekday >= 5) & (8 <= time_hour <= 12):
    time_based_param_set = ['home_beach_driving']
# Weekend Evening
elif (weekday >= 5) & (12 <= time_hour <= 20):
    time_based_param_set = ['beach_home_driving']
# WeekDay Morning
elif (weekday < 5) & (5 <= time_hour <= 10):
    time_based_param_set = ['home_work_driving', 'home_work_transit', 'home_bxbark_driving', 'bxbark_work_driving']
# WeekDay Evening
elif (weekday < 5) & (15 <= time_hour <= 22):
    time_based_param_set = ['work_home_driving', 'work_home_transit', 'work_bxbark_driving', 'bxbark_home_driving']
else: 
    time_based_param_set = ['home_beach_driving', 'beach_home_driving']
    print('Uhoh')

print("Params: ", time_based_param_set)
print('Weekday: ', weekday, "Time Hour: ", time_hour)

Params:  ['beach_home_driving']
Weekday:  5 Time Hour:  18


In [18]:
try:
    db_updater = rd.GetRawAPI()
    print("finished s1")
    db_updater.execute(param_names = time_based_param_set)
    print(db_updater.updated_params_df)
    print("finished s2")
    
    driving_parser = parser.RunParser('driving', 'parsed_driving')
    print("finished s3")
    driving_parser.execute()
    print("finished s4")
    transit_parser = parser.RunParser('transit', 'parsed_transit')
    print("finished s5")
    transit_parser.execute()
    

    
    hp_error = None
except Exception as e:
    hp_error = str(e)
    print("Error: ", hp_error)

Let's get started
finished s1
Added to /home/malcolm/WorkTransit/data/transit.db
                           origin                destination     mode  \
0  Orchard Beach, Bronx, NY 10464  120 Benchley Pl, Bronx NY  driving   

               departure_time  \
0  2022-06-24 17:17:06.331790   

                                        Raw Response        Date   Time  
0  [{'bounds': {'northeast': {'lat': 40.874488100...  2022-06-24  17:17  
finished s2
finished s3
Error:  list index out of range


## Emails

In [19]:
sys.path.append('/home/malcolm/EmailSender/')
from EmailSender import EmailSender as ES

In [20]:
today_str = str(datetime.datetime.now().date())
if isinstance(hp_error, str):
    error_email = {}
    error_email['Subject'] = "Error in Google Directions "+today_str
    error_email['Body'] = """
        There was an error in Google Directions data pull. 

        Stack Trace: 

        """ + hp_error

    email_sender = ES(**error_email)
    email_sender.execute()
    

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials j13-20020a05620a288d00b006a3325fd985sm2876010qkp.13 - gsmtp')

In [ ]:
today_str = str(datetime.datetime.now().date())

if isinstance(hp_error, str):
    subject = "Error in Google Directions "+today_str
    message["Subject"] = subject

    
    body = """
        There was an error in Google Directions data pull. 

        Coming from Digital Ocean Droplet 

        Stack Trace: 

        """ + hp_error
    
    text = MIMEText(body, "plain")
    message.attach(text)
             
    print("Starting email send")
    with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
        server.login(gmail_login, pw)
        message_str = message.as_string()
        server.sendmail(gmail_login, reciever_email, message_str)


## Scraper 1

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[7]:


import requests
import pickle
import json
import os
import sys
import googlemaps
import datetime 
import sqlite3
import pandas as pd
from ast import literal_eval

locations = dict(home = "120 Benchley Pl, Bronx NY",
work = "809 Washington Street",
bxbark = "830 Pelham Pkwy, Pelham Manor, NY 10803",
orchard_beach = "Orchard Beach, Bronx, NY 10464" ,
                )

home = "120 Benchley Pl, Bronx NY"
work = "809 Washington Street"
bxbark = "830 Pelham Pkwy, Pelham Manor, NY 10803"
orchard_beach = "Orchard Beach, Bronx, NY 10464"
now = datetime.datetime.now() + datetime.timedelta(minutes=3)

parameter_sets = dict(
    home_work_driving = {'origin' : home
                     , 'destination' : work
                     , 'mode' : "driving"
                     , 'departure_time' : now}, 
    home_work_transit = {'origin' : home
                     , 'destination' : work
                     , 'mode' : "transit"
                     , 'departure_time' : now}, 
    work_home_transit = {'origin' : work
                     , 'destination' : home
                     , 'mode' : "transit"
                     , 'departure_time' : now}, 
    work_home_driving = {'origin' : work
                     , 'destination' : home
                     , 'mode' : "driving"
                     , 'departure_time' : now}, 
    home_bxbark_driving = {'origin' : home
                     , 'destination' : bxbark
                     , 'mode' : "driving"
                     , 'departure_time' : now}, 
    bxbark_work_driving = {'origin' : bxbark
                     , 'destination' : work
                     , 'mode' : "driving"
                     , 'departure_time' : now}, 
    work_bxbark_driving = {'origin' : work
                     , 'destination' : bxbark
                     , 'mode' : "driving"
                     , 'departure_time' : now}, 
    bxbark_home_driving = {'origin' : work
                     , 'destination' : bxbark
                     , 'mode' : "driving"
                     , 'departure_time' : now}, 
    home_beach_driving = {'origin' : home
                     , 'destination' : orchard_beach
                     , 'mode' : "driving"
                     , 'departure_time' : now}, 
    beach_home_driving = {'origin' : orchard_beach
                     , 'destination' : home
                     , 'mode' : "driving"
                     , 'departure_time' : now}, 
) 

In [2]:
class GetRawAPI():
    
    def __init__(self):
        print("Let's get started")
        pass

    @staticmethod 
    def create_param_sets(param_names):
        param_set_list = [parameter_sets[x] for x in param_names]
        return(param_set_list)

    @staticmethod
    def get_gmap_directions(param_set_list, google_key_path):
        # Start google client 
        with open(google_key_path, 'rb') as hnd:
            api_key = pickle.load(hnd)
        gmaps = googlemaps.Client(key=api_key)

        # Make API Calls 
        updated_params_list = []
        for param_set in param_set_list:
            param_set2 = param_set.copy()
            temp_result = gmaps.directions(**param_set)

            date = str(param_set2['departure_time'].date())
            time = str(param_set2['departure_time'].strftime('%H:%M'))
            param_set2.update({'Raw Response' : temp_result
                              , 'Date' : date
                              , 'Time' : time})
            updated_params_list.append(param_set2)

        # Fix up dataframe 
        updated_params_df = pd.DataFrame(updated_params_list)
        updated_params_df['Raw Response'] = updated_params_df['Raw Response'].astype(str)
        updated_params_df['departure_time'] = updated_params_df['departure_time'].astype(str)
        return(updated_params_df)

    @staticmethod
    def update_db(updated_params_df, db_path, db_file_name):
        # Ensure paths and files are ready 
        os.makedirs(db_path, exist_ok=True)
        if db_path[-1] != '/':
            db_path = db_path + '/'
        db_loc = db_path + db_file_name
        
        # Create SQL Connection and append to db
        con = sqlite3.connect(db_loc)
        updated_params_df.to_sql('Raw_Responses', con, index=False, if_exists='append' )
        
        # Close connection
        con.commit()
        con.close()
        print(f"Added to {db_loc}")
    
    def execute(self
                , param_names = ['home_work_driving', 'home_work_transit', 'home_bxbark_driving', 'bxbark_work_driving']
                , google_key_path = '/home/malcolm/credentials/google_transit_api.pkl'
                , db_path = '/home/malcolm/WorkTransit/data/'
                , db_file_name = 'transit.db'
               ):
        self.param_set_list = self.create_param_sets(param_names)
        self.updated_params_df = self.get_gmap_directions(self.param_set_list, google_key_path = google_key_path)
        self.update_db(self.updated_params_df, db_path = db_path, db_file_name = db_file_name)

        

# In[11]:
## Going to work params
# param_names = ['home_work_driving', 'home_work_transit', 'home_bxbark_driving', 'bxbark_work_driving']
## Coming from work params 
# param_names = ['work_home_driving', 'work_home_transit', 'work_bxbark_driving', 'bxbark_home_driving']

# db_updater = GetRawAPI()
# db_updater.execute()

# print(db_updater.updated_params_df)

In [5]:
db_updater = GetRawAPI()


Let's get started


In [6]:
!date

Fri Jun 24 17:13:31 EDT 2022


In [7]:
param_names = ['home_work_driving', 'home_work_transit', 'home_bxbark_driving', 'bxbark_work_driving']
google_key_path = '/home/malcolm/credentials/google_transit_api.pkl'
db_path = '/home/malcolm/WorkTransit/data/'
db_file_name = 'transit.db'

In [8]:
db_updater.param_set_list = db_updater.create_param_sets(param_names)
db_updater.param_set_list

[{'origin': '120 Benchley Pl, Bronx NY',
  'destination': '809 Washington Street',
  'mode': 'driving',
  'departure_time': datetime.datetime(2022, 6, 24, 17, 16, 22, 787775)},
 {'origin': '120 Benchley Pl, Bronx NY',
  'destination': '809 Washington Street',
  'mode': 'transit',
  'departure_time': datetime.datetime(2022, 6, 24, 17, 16, 22, 787775)},
 {'origin': '120 Benchley Pl, Bronx NY',
  'destination': '830 Pelham Pkwy, Pelham Manor, NY 10803',
  'mode': 'driving',
  'departure_time': datetime.datetime(2022, 6, 24, 17, 16, 22, 787775)},
 {'origin': '830 Pelham Pkwy, Pelham Manor, NY 10803',
  'destination': '809 Washington Street',
  'mode': 'driving',
  'departure_time': datetime.datetime(2022, 6, 24, 17, 16, 22, 787775)}]

In [9]:
db_updater.updated_params_df = db_updater.get_gmap_directions(db_updater.param_set_list, google_key_path = google_key_path)

In [10]:
db_updater.update_db(db_updater.updated_params_df, db_path = db_path, db_file_name = db_file_name)

Added to /home/malcolm/WorkTransit/data/transit.db


/home/malcolm/main/lib/python3.6/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


## Parser

In [31]:
import pandas as pd 
import sqlite3 
import numpy as np 
import datetime
import sys 
import os 
from ast import literal_eval
import logging 

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s :: %(levelname)s :: %(message)s')


def parse_time(x):
    return(str(datetime.datetime.utcfromtimestamp(x)))

def parse_duration(x):
    return(np.round(x/3600.0, 2))

def parse_distance(x):
    return(np.round(x/1609.34, 2))

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def summarize_leg_step(step):
    out = {}
    out['distance_mi'] = parse_distance(step['distance']['value'])
    out['duration_hr'] = parse_duration(step['duration']['value'])
    out['instructions'] = remove_html_tags(step['html_instructions'])

    out['distance_text'] = step['distance']['text']
    out['duration_text'] = step['duration']['text']

    out_series = pd.Series(out)
    return(out_series)


def consolidate_steps(driving_steps_dict):
    concise_steps = pd.DataFrame([summarize_leg_step(x) for x in driving_steps_dict])
    concise_steps = concise_steps.reset_index().rename({'index':'Step #'}, axis=1)
    concise_steps = concise_steps.sort_values('duration_hr', ascending=False)
    return(concise_steps)

def parse_driving_details(driving_resp):

    legs = driving_resp['legs'][0]
    concise_steps = consolidate_steps(legs['steps'])

    driving_details = {}
    driving_details['summary'] = driving_resp['summary']
    driving_details['duration_hr'] = parse_duration(legs['duration']['value'])
    driving_details['duration_text'] = legs['duration']['text']

    driving_details['duration_in_traffic_hr'] = parse_duration(legs['duration_in_traffic']['value'])
    driving_details['duration_in_traffic_text'] = legs['duration_in_traffic']['text']

    driving_details['extra time in traffic_min'] = np.round((driving_details['duration_in_traffic_hr'] - driving_details['duration_hr'])*60.0, 2)

    driving_details['distance_mi'] = parse_distance(legs['distance']['value'])


    driving_details['1st Longest Step (time)'] = str(concise_steps.iloc[0].to_dict())
    driving_details['2nd Longest Step (time)'] = str(concise_steps.iloc[1].to_dict())
    driving_details['3rd Longest Step (time)'] = str(concise_steps.iloc[2].to_dict())

    return(pd.Series(driving_details))


def parse_transit_details(transit_resp):
    transit_details = {}
    legs = transit_resp['legs'][0]
    transit_steps = [x for x in legs['steps'] if x['travel_mode'] == 'TRANSIT']
    lines = [x['transit_details']['line'].get('short_name') for x in transit_steps]
    transit_details['departure_time'] = parse_time(legs['departure_time']['value'])
    transit_details['arrival_time'] = parse_time(legs['arrival_time']['value'])
    transit_details['duration_hr'] = parse_duration(legs['duration']['value'])
    transit_details['duration_txt'] = legs['duration']['text']
    transit_details['distance_mi'] = parse_distance(legs['distance']['value'])
    transit_details['transit lines'] = lines
    transit_details['transit departure_time'] = [parse_time(x['transit_details']['departure_time']['value']) 
                                         for x in transit_steps]
    transit_details['transit arrival_time'] = [parse_time(x['transit_details']['arrival_time']['value']) 
                                         for x in transit_steps]
    transit_details['transit durations_hr'] = [parse_duration(x.get('duration').get('value')) 
                                            for x in transit_steps]
    transit_details = {k:str(v) if type(v) == list else v 
                      for k, v in transit_details.items()}
    return(pd.Series(transit_details))

In [32]:
class RunParser():
    
    def __init__(self, mode, parsed_tbl_name):
        self.mode = mode
        if mode not in ['driving', 'transit']:
            raise ValueError()
        
        self.parsed_tbl_name = parsed_tbl_name
        if parsed_tbl_name not in ['parsed_transit', 'parsed_driving']:
            raise ValueError()
        
        if self.mode == 'transit':
            self.col_name = 'departure_time_y'
        elif self.mode == 'driving':
            self.col_name = 'departure_time'
        else:
            logger.warn("Mode not in [transit, driving] .. not sure departure time colname ")
                
    def create_con(self, db_path):
        self.con = sqlite3.connect(db_path)
        self.cursor = self.con.cursor()
        self.cursor.execute('SELECT name FROM sqlite_master WHERE type=\'table\' ORDER BY name')
        tables = self.cursor.fetchall()
        tables = [x[0] for x in tables]
        logger.debug("Tables: ", tables)
    
    def get_latest_date(self):
        sql = f"select max(date) as date, max(time) as time from {self.parsed_tbl_name}"
        date, time = self.cursor.execute(sql).fetchall()[0]
        logger.debug(f"Latest date for {self.parsed_tbl_name} is {date} {time}")
        return(date, time)
    
    def get_raw_responses_df(self, latest_date, latest_time):
        raw_df = pd.read_sql(f"""
            select * from raw_responses 
            where Date >= '{latest_date}'
            and Time >= '{latest_time}'
            and mode = '{self.mode}'
            """, self.con)
        logger.debug("Raw Response df has ", raw_df.shape[0], " rows")
        return(raw_df)
    
    def get_latest_date2(self):
        sql = f"select max({self.col_name}) as latest_date_time from {self.parsed_tbl_name}"
        latest_date_time = self.cursor.execute(sql).fetchall()[0][0]
        logger.debug(f"Latest date for {self.parsed_tbl_name} is {latest_date_time}")
        return(latest_date_time)
    
    def get_raw_responses_df2(self, latest_date_time):
        raw_df = pd.read_sql(f"""
            select * from raw_responses 
            where departure_time >= '{latest_date_time}'
            and mode = '{self.mode}'
            """, self.con)
        logger.debug("Raw Response df has ", raw_df.shape[0], " rows")
        return(raw_df)
    
    def parse_raw_df(self, raw_df):
        logger.debug("Starting parse raw")
        if self.mode =='transit':
            raw_df = raw_df[raw_df['Raw Response'] != '[]']
            transit_df_parsed = raw_df['Raw Response']\
                .apply(lambda x: parse_transit_details(literal_eval(x)[0]))
            return(transit_df_parsed)
        elif self.mode == 'driving':
            raw_df = raw_df[raw_df['Raw Response'] != '[]']
            driving_df_parsed = raw_df['Raw Response']\
                .apply(lambda x: parse_driving_details(literal_eval(x)[0]))
            return(driving_df_parsed)
        else:
            logger.info("mode not transit or driving.... Unexpected returning empty df")
            return(pd.DataFrame())

    def merge(self, raw_df, parsed_df):
        full = pd.merge(parsed_df, raw_df
             , left_index=True, right_index=True)
        return(full)
    
    def update_table(self, full_df):
        full_df.to_sql(self.parsed_tbl_name, self.con, index=False, if_exists='append')
        self.con.commit()
        logger.debug(self.parsed_tbl_name, " was updated with ", full_df.shape[0], " rows")

    def execute(self
               , db_path = '/home/malcolm/WorkTransit/data/transit.db'
               , parsed_table_name = 'parsed_transit', column_name='date'):
        self.create_con(db_path)
#         self.latest_date, self.latest_time = self.get_latest_date()
#         self.raw_df = self.get_raw_responses_df(self.latest_date, self.latest_time)
        # Testing
        self.latest_date_time = self.get_latest_date2()
        self.raw_df = self.get_raw_responses_df2(self.latest_date_time)
        self.parsed_df = self.parse_raw_df(self.raw_df)
        self.full_df = self.merge(self.raw_df, self.parsed_df)
        self.update_table(self.full_df)

In [33]:
# driving_parser = RunParser('driving', 'parsed_driving')
# print("finished s3")
# driving_parser.execute()
# print("finished s4")
transit_parser = RunParser('transit', 'parsed_transit')
print("finished s5")
transit_parser.execute()

finished s4
finished s5


In [ ]:
driving_parser.raw_df

In [34]:
dir(transit_parser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'col_name',
 'con',
 'create_con',
 'cursor',
 'execute',
 'full_df',
 'get_latest_date',
 'get_latest_date2',
 'get_raw_responses_df',
 'get_raw_responses_df2',
 'latest_date_time',
 'merge',
 'mode',
 'parse_raw_df',
 'parsed_df',
 'parsed_tbl_name',
 'raw_df',
 'update_table']

In [35]:
transit_parser.parsed_df

,departure_time,arrival_time,duration_hr,duration_txt,distance_mi,transit lines,transit departure_time,transit arrival_time,transit durations_hr
0,2022-03-30 13:20:39,2022-03-30 14:50:56,1.50,1 hour 30 mins,15.89,"['5', 'L']","['2022-03-30 13:43:30', '2022-03-30 14:42:30']","['2022-03-30 14:33:30', '2022-03-30 14:45:00']","[0.83, 0.04]"
2,2022-03-30 13:56:35,2022-03-30 15:22:56,1.44,1 hour 26 mins,15.62,"['Bx28', '5', '4', 'L']","['2022-03-30 14:03:25', '2022-03-30 14:21:30',...","['2022-03-30 14:12:37', '2022-03-30 14:47:00',...","[0.15, 0.42, 0.34, 0.04]"
3,2022-03-30 21:04:21,2022-03-30 22:24:21,1.33,1 hour 20 mins,17.71,"['L', '5', 'BxM7']","['2022-03-30 21:10:30', '2022-03-30 21:18:50',...","['2022-03-30 21:14:00', '2022-03-30 21:36:20',...","[0.06, 0.29, 0.43]"
4,2022-03-30 21:20:21,2022-03-30 22:39:21,1.32,1 hour 19 mins,17.70,"['L', '4', 'BxM7']","['2022-03-30 21:26:30', '2022-03-30 21:33:56',...","['2022-03-30 21:30:00', '2022-03-30 21:51:26',...","[0.06, 0.29, 0.43]"
5,2022-03-30 21:37:51,2022-03-30 22:56:21,1.31,1 hour 19 mins,17.71,"['L', '5', 'BxM7']","['2022-03-30 21:44:00', '2022-03-30 21:51:30',...","['2022-03-30 21:47:30', '2022-03-30 22:09:00',...","[0.06, 0.29, 0.43]"
...,...,...,...,...,...,...,...,...,...
1487,2022-06-24 13:53:36,2022-06-24 15:22:41,1.48,1 hour 29 mins,16.06,"['Bx28', '5', '2']","['2022-06-24 14:00:25', '2022-06-24 14:21:30',...","['2022-06-24 14:09:37', '2022-06-24 14:47:00',...","[0.15, 0.42, 0.39]"
1488,2022-06-24 21:04:21,2022-06-24 22:24:21,1.33,1 hour 20 mins,17.70,"['L', '5', 'BxM7']","['2022-06-24 21:10:30', '2022-06-24 21:19:03',...","['2022-06-24 21:14:45', '2022-06-24 21:36:33',...","[0.06, 0.29, 0.43]"
1489,2022-06-24 21:18:32,2022-06-24 22:51:26,1.55,1 hour 33 mins,17.47,"['Bx23', '6', 'L']","['2022-06-24 21:21:33', '2022-06-24 21:44:30',...","['2022-06-24 21:36:00', '2022-06-24 22:37:00',...","[0.24, 0.88, 0.04]"
1490,2022-06-24 21:20:21,2022-06-24 22:39:21,1.32,1 hour 19 mins,17.70,"['L', '4', 'BxM7']","['2022-06-24 21:26:30', '2022-06-24 21:34:24',...","['2022-06-24 21:30:45', '2022-06-24 21:51:54',...","[0.06, 0.29, 0.43]"


In [37]:
transit_parser.parsed_df[transit_parser.parsed_df['transit lines'].isna()]

,departure_time,arrival_time,duration_hr,duration_txt,distance_mi,transit lines,transit departure_time,transit arrival_time,transit durations_hr
